In [1]:
import pandas as pd
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [3]:
base_costa = pd.read_csv(r'C:\Users\LENOVO\Desktop\base_costa.txt', sep = ",", engine = "python")

In [4]:
base2_costa = base_costa[['SUSCNITC','SERVDESC','SESUCUSA','SESUCICL','SESULOCA',
                                  'TIPIFICACION_CLIENTE','CALIFICACION_VECTOR_SERVICIO',
                                  'CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR',
                                  'ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC']]
base3_costa = base2_costa.dropna()

In [6]:
tmp = base3_costa[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
tmp.columns

group_nit = tmp.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()
group_nit = group_nit.rename(columns={"CALIFICACION_VECTOR_SERVICIO":"PROM_CALIFICACION"})

In [8]:
base4_costa = pd.merge(base3_costa, group_nit, how='left', on=['SUSCNITC'])

In [9]:
base4_costa['y'] = pd.cut(x=base4_costa['PROM_CALIFICACION'],
                            bins=[-1,50,76,100],
                            labels=['No Pago','Pago Inoportuno','Pago'])

In [11]:
base5_costa = base4_costa[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE',
                           'CALIFICACION_VECTOR_SERVICIO','CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO',
                           'DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN',
                           'PROM_SUSC','y']]

In [12]:
names = base5_costa.columns

for x in names:
    globals()['FREQ_%s' % x]= base5_costa[x].value_counts().rename_axis([x]).reset_index(name='F_{x}'.format(x=x))

In [14]:
freq_costa = base5_costa.merge(FREQ_SERVDESC,on="SERVDESC").merge(FREQ_SESUCUSA,on='SESUCUSA').merge(FREQ_SESUCICL,on='SESUCICL').merge(FREQ_SESULOCA,on='SESULOCA').merge(FREQ_TIPIFICACION_CLIENTE,on='TIPIFICACION_CLIENTE').merge(FREQ_CANAL_ENVIO,on='CANAL_ENVIO').merge(FREQ_SEGMENTACION,on='SEGMENTACION').merge(FREQ_ESTRATO_AGRUPADO,on='ESTRATO_AGRUPADO').merge(FREQ_DEPARTAMENTO_AGR,on='DEPARTAMENTO_AGR').merge(FREQ_CANAL_PAGO,on='CANAL_PAGO')

In [15]:
freq_costa2 = freq_costa[['F_SERVDESC','F_SESUCUSA','F_SESUCICL','F_SESULOCA','F_TIPIFICACION_CLIENTE',
                          'F_CANAL_ENVIO','F_SEGMENTACION','F_ESTRATO_AGRUPADO','F_DEPARTAMENTO_AGR',
                          'F_CANAL_PAGO']]

In [16]:
freq_costa2['AVG_FREQ'] = freq_costa2.mean(axis=1)

C:\Users\LENOVO\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
n, m = freq_costa2.shape
n_reg_del = round((0.05 * n) /100)
outliers = freq_costa2.nsmallest(n_reg_del, 'AVG_FREQ')
outliers_idx = outliers.index
outliers_idx = outliers_idx.sort_values(ascending=False)

In [18]:
for x in outliers_idx:
    base5_costa = base5_costa.drop([x])

In [19]:
dataTypeSeries = base5_costa.dtypes
dataTypeSeries

cat = base5_costa.select_dtypes(include=['object','category'])
cuanti = base5_costa.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))
base5_costa = cuanti.join(cat)

In [20]:
X = base5_costa[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE','CANAL_ENVIO',
               'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO',
               'SESUSAPE','SESUSAAN','PROM_SUSC']]

y = base5_costa['y']

In [21]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

In [22]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.783898  0.781468  0.782681   31286.0   31189.0  0.930538
1             0.763788  0.841671  0.800841   62705.0   69099.0  0.889884
2             0.692795  0.610029  0.648783   52709.0   46412.0  0.802157
avg / total   0.742569  0.745603  0.742334  146700.0  146700.0  0.885422,              precision    recall  f1-score   support      pred       AUC
0             0.797087  0.757368  0.776720   31286.0   29727.0  0.923487
1             0.757257  0.840380  0.796656   62705.0   69588.0  0.883045
2             0.681334  0.612514  0.645094   52709.0   47385.0  0.797376
avg / total   0.738472  0.740804  0.737948  146700.0  146700.0  0.880701,              precision    recall  f1-score   support      pred       AUC
0             0.794790  0.758742  0.776348   31286.0   29867.0  0.919746
1             0.752120  0.841432  0.794273   62705.0   70151.0  0.890904
2             0.685575  0.607183  0.644002   527

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[24086,  1542,  5657],
       [ 1241, 52466,  8997],
       [ 4897, 13157, 34656]], dtype=int64)